In [73]:
#The dataset is not in usable form like csv, gonna use pandas to do it

In [74]:
import pyprind
import pandas as pd
import os

In [ ]:
pbar = pyprind.ProgBar(50000)

In [ ]:
labels = {'pos':1, 'neg':0}

In [77]:
df = pd.DataFrame()

In [ ]:
for s in ('test','train'):
    for l in ('pos','neg'):
        path = 'C:/Users/Vivek/Desktop/aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path,file),'r') as infile:
                txt = infile.read()
            df = df.append([[txt,labels[l]]], ignore_index = True)
            pbar.update()

In [79]:
df.columns = ['review','sentiment']

In [80]:
import numpy as np
np.random.seed(0)

In [81]:
df = df.reindex(np.random.permutation(df.index))

In [ ]:
df.to_csv('C:/Users/Vivek/Desktop/aclImdb/movdata.csv')

In [83]:
#df = pd.read_csv('C:/Users/Vivek/Desktop/aclImdb/movdata.csv')
df.head(10)

,review,sentiment
11841,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
19602,OK... so... I really like Kris Kristofferson a...,0
45519,"***SPOILER*** Do not read this, if you think a...",0
25747,hi for all the people who have seen this wonde...,1
42642,"I recently bought the DVD, forgetting just how...",0
31902,Leave it to Braik to put on a good show. Final...,1
30346,Nathan Detroit (Frank Sinatra) is the manager ...,1
12363,"To understand ""Crash Course"" in the right cont...",1
32490,I've been impressed with Chavez's stance again...,1
26128,This movie is directed by Renny Harlin the fin...,1


In [84]:
#cleaning data

In [85]:
import re

In [86]:
def preprocessor(text):
    text = re.sub('<[^>]*>','', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+',' ', text.lower())
    #text = re.join(emoticons).replace('-','')
    return text

In [87]:
df['review'] = df['review'].apply(preprocessor)

In [88]:
df.head(10)

,review,sentiment
11841,in 1974 the teenager martha moxley maggie grac...,1
19602,ok so i really like kris kristofferson and his...,0
45519,spoiler do not read this if you think about w...,0
25747,hi for all the people who have seen this wonde...,1
42642,i recently bought the dvd forgetting just how ...,0
31902,leave it to braik to put on a good show finall...,1
30346,nathan detroit frank sinatra is the manager of...,1
12363,to understand crash course in the right contex...,1
32490,i ve been impressed with chavez s stance again...,1
26128,this movie is directed by renny harlin the fin...,1


In [89]:
#Splitting the data into training and testing data
X_train = df.loc[:30000 , 'review'].values
y_train = df.loc[:30000, 'sentiment'].values
X_test = df.loc[20000:, 'review'].values
y_test = df.loc[20000:, 'sentiment'].values

In [90]:
#Now that data is clean, it is broken down into vectors, don't know how it happens but its just transformation

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

In [92]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000)

In [93]:
features = vectorizer.fit_transform(X_train)

In [94]:
print(features.shape)

(43987, 5000)


In [95]:
from sklearn.naive_bayes import MultinomialNB

In [96]:
nbmodel = MultinomialNB()

In [97]:
train_pred = nbmodel.fit(features, y_train)

In [98]:
print(train_pred)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [99]:
from sklearn import metrics

In [100]:
test_set = vectorizer.fit_transform(X_test)

In [101]:
predictions = nbmodel.predict(test_set)
print(predictions)

[1 1 1 ..., 0 1 1]


In [102]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, predictions, pos_label=1)

In [103]:
print("Multinomial naive bayes AUC: {0}".format(metrics.auc(fpr, tpr)))

Multinomial naive bayes AUC: 0.52633386553
